In [8]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import wfdb

from collections import Counter

## Data Pre-processing

In [9]:
# get all the record names in a list
namelist = []
namelist = namelist + list(range(100,125,1)) + list(range(200,235,1))
for num in [110,120,204,206,211,216,218,224,225,226,227,229]:
    namelist.remove(num)

len(namelist)

48

In [10]:
def get_window (signals, annotation, sec):
    """
    this function gives a sec-seconds window (sec seconds before, sec seconds after the annotation mark)
    of the ECG signals and assign value 1 if it's PVC beat and 0 otherwise.
    parameter: signals: numpy array containing heart beat record values
               annotation: wfdb.annotation object containing heart beat annotations
               sec: positive integer number indicating the half-width of the window
    return: two lists
            siglist: a list of lists of length 360*2*sec 
            annlist: a list containing 1 if PVC beat, 0 otherwise
    """
    siglist = []
    annlist = []
    
    #loop through the annotation.symbol list
    for i in range(len(annotation.symbol)):
        timestamp = annotation.sample[i] #get the timestamp
        
        #test if that timestamp can have sec seconds before and after window
        windowStart = timestamp - sec*annotation.fs
        windowEnd = timestamp + sec*annotation.fs
        if windowStart >= 0 & windowEnd <= len(signals):
            if annotation.symbol[i] == 'V':
                # check if the length of this strip is 360*2*sec
                if len(signals[windowStart:windowEnd,].flatten().tolist()) == 2*sec*annotation.fs:
                    siglist.append(signals[windowStart:windowEnd,].flatten().tolist())
                    annlist.append(1)
                else:
                    continue
            else:
                # check if the length of this strip is 360*2*sec
                if len(signals[windowStart:windowEnd,].flatten().tolist()) == 2*sec*annotation.fs:
                    siglist.append(signals[windowStart:windowEnd,].flatten().tolist())
                    annlist.append(0)
        else:
            continue
    
    return siglist, annlist

In [11]:
# loop through all record to get all the 10-seconds window signal list and annotation list
# this could be the training dataset for the neural network model
# this takes several minutes to run

ECG_signals = []
PVC_annotations = []

for record in namelist:
    signals, fields = wfdb.rdsamp(str(record), sampfrom=0, sampto='end', channels=[1], pb_dir='mitdb')
    annotation = wfdb.rdann(str(record), 'atr', sampfrom=0, sampto=None, shift_samps=True, pb_dir='mitdb')
    
    signal_list, annotation_list = get_window(signals, annotation, 5)
    
    ECG_signals = ECG_signals + signal_list
    PVC_annotations = PVC_annotations + annotation_list
    
    print('record', record, 'is done.')

record 100 is done.
record 101 is done.
record 102 is done.
record 103 is done.
record 104 is done.
record 105 is done.
record 106 is done.
record 107 is done.
record 108 is done.
record 109 is done.
record 111 is done.
record 112 is done.
record 113 is done.
record 114 is done.
record 115 is done.
record 116 is done.
record 117 is done.
record 118 is done.
record 119 is done.
record 121 is done.
record 122 is done.
record 123 is done.
record 124 is done.
record 200 is done.
record 201 is done.
record 202 is done.
record 203 is done.
record 205 is done.
record 207 is done.
record 208 is done.
record 209 is done.
record 210 is done.
record 212 is done.
record 213 is done.
record 214 is done.
record 215 is done.
record 217 is done.
record 219 is done.
record 220 is done.
record 221 is done.
record 222 is done.
record 223 is done.
record 228 is done.
record 230 is done.
record 231 is done.
record 232 is done.
record 233 is done.
record 234 is done.


In [13]:
len(ECG_signals)

111985

In [14]:
len(PVC_annotations)

111985

In [15]:
# there are 7112 PVC 10-seconds window, 105186 non-PVC 10-seconds window
Counter(PVC_annotations)

Counter({0: 104886, 1: 7099})

In [25]:
# get the index in PVC_annotations for normal beats

normal_index = [i for i in range(len(PVC_annotations)) if PVC_annotations[i]==0]
len(normal_index)

104886

In [21]:
# get the index in PVC_annotations for PVC beats

PVC_index = [i for i in range(len(PVC_annotations)) if PVC_annotations[i]==1]
len(PVC_index)

7099

In [30]:
# randomly select 7099 normal beats

import random

random.seed(77)
small_normal_index = random.sample(normal_index, 7099)

In [31]:
len(small_normal_index)

7099

In [32]:
small_normal_index[:10]

[111839, 33939, 44009, 26542, 32244, 25993, 15648, 39353, 66405, 77501]

In [33]:
# concatenate 7099 normal beats index with 7099 PVC beats index
# as small data index

small_data_index = small_normal_index + PVC_index
len(small_data_index)

14198

In [34]:
# generate samll dataset

small_signals = []
for i in small_data_index:
    small_signals.append(ECG_signals[i])
    
small_annotations = []
for i in small_data_index:
    small_annotations.append(PVC_annotations[i])

In [35]:
len(small_signals)

14198

In [36]:
len(small_annotations)

14198

In [37]:
# write ECG signals data to csv file

import csv

with open('MIT-BIH-small.csv', 'w') as f:
    f_csv = csv.writer(f)
    f_csv.writerows(small_signals)

In [38]:
# write the PVC annotation data to csv file

with open('MIT-BIH-Annotation-small.csv', 'w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(small_annotations)

## Classification Models

In [39]:
# transform data into numpy array

X = np.array(small_signals)
y = np.array(small_annotations)

In [41]:
X.shape

(14198, 3600)

In [42]:
y.shape

(14198,)

In [44]:
# split train and test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [46]:
X_train.shape

(9938, 3600)

In [47]:
y_train.shape

(9938,)

In [48]:
X_test.shape

(4260, 3600)

In [49]:
y_test.shape

(4260,)

In [50]:
# the ratio of PVC beats in training set and test set

Counter(y_train)

Counter({0: 4983, 1: 4955})

In [51]:
Counter(y_test)

Counter({0: 2116, 1: 2144})

In [52]:
# baseline model accuracy

2144/(2116+2144)

0.5032863849765258